## Data import

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os, shutil
!ls "/content/gdrive/My Drive/"

 8-2.mov
 Bayesian
 bigdata
 bq-results-20190920-004228-5xi034liest1
 bq-results-20190920-012138-er5eg3s9lxkj
 bq-results-20190920-012622-6g3qu1tkjun8
 bq-results-20191112-190906-w6n93g514345
 CAMELYON16
 checkpoints
'Chu Yu-resume.pdf'
'Colab Notebooks'
'data science final scratch.gdoc'
'Download eTranscripts ordered from your school (Windows) (1).gdoc'
 drug.csv
'epi assignment 5.gdoc'
 measure.csv
 my_model.h5
 observation.csv
'Phar stats final SAP.gdoc'
 scudata
 sculpture


In [0]:
obs_df = pd.read_csv('/content/gdrive/My Drive/observation.csv')

In [0]:
obs_df.head(5)

,person_id,visit_occurrence_id,year_of_birth,gender_source_value,ethnicity_concept_id,person_source_value,month_of_birth,day_of_birth,ethnicity_source_value,location_id,race_concept_id,gender_concept_id,birth_datetime,race_source_value,death_date,death_datetime,death_type_concept_id,cause_source_concept_id,provider_id,visit_start_date,care_site_id,visit_source_value,visit_end_date,visit_start_datetime,visit_concept_id,visit_type_concept_id,visit_end_datetime,value_as_concept_id,observation_date,observation_source_concept_id,value_as_string,observation_concept_id,observation_id,observation_source_value,observation_source_concept_i,observation_datetime
0,5,4213630.0,1935,1,38003564,000308435E3E5B76,9,1,1,8,8557.0,8507,1935-09-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,68338.0,2009-01-09T00:00:00.000Z,2870.0,5.421323e+14,2009-01-09T00:00:00.000Z,2009-01-09T00:00:00.000Z,9202.0,44818517.0,2009-01-09T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41,1656926.0,1941,2,38003564,001F988D97A7759B,2,1,1,59,8527.0,8532,1941-02-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,179231.0,2008-10-09T00:00:00.000Z,42872.0,8.874434e+14,2008-10-09T00:00:00.000Z,2008-10-09T00:00:00.000Z,0.0,44818517.0,2008-10-09T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62,3963163.0,1943,1,38003564,002B6203FF086ABA,9,1,1,35,8516.0,8507,1943-09-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,189926.0,2008-06-30T00:00:00.000Z,86906.0,8.872734e+14,2008-06-30T00:00:00.000Z,2008-06-30T00:00:00.000Z,0.0,44818517.0,2008-06-30T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103,58348.0,1935,1,38003564,0050CF9059ABFF59,6,1,1,134,8657.0,8507,1935-06-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,42069.0,2009-07-25T00:00:00.000Z,5573.0,8.874534e+14,2009-07-25T00:00:00.000Z,2009-07-25T00:00:00.000Z,0.0,44818517.0,2009-07-25T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125,1886636.0,1926,1,38003564,0064148837D38095,5,1,1,154,8657.0,8507,1926-05-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,54829.0,2009-01-13T00:00:00.000Z,30634.0,8.870134e+14,2009-01-13T00:00:00.000Z,2009-01-13T00:00:00.000Z,0.0,44818517.0,2009-01-13T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#drug_df = drug_df.apply(lambda x:x.fillna(x.value_counts().index[0]))
obs_df = obs_df[obs_df['visit_start_datetime'] == obs_df.groupby('person_id')['visit_start_datetime'].transform('max')]

## data preprocessing

In [0]:
persondrop = ['person_source_value','ethnicity_source_value', 'gender_source_value','location_id', 'race_source_value', 'care_site_id']
visitdrop = ['visit_source_value','visit_start_date','visit_end_date','visit_end_datetime', 'provider_id', 'value_as_concept_id']
drugdrop = [ 'observation_source_value', "observation_date", 'observation_source_concept_i']

In [0]:
obs_df=obs_df.drop(columns=persondrop)
obs_df=obs_df.drop(columns=visitdrop)
obs_df=obs_df.drop(columns=drugdrop)

In [0]:
obs_df=obs_df.drop(columns=['observation_id', 'provider_id', 'visit_type_concept_id', ''])

In [0]:
obs_df.loc[pd.isnull(obs_df['death_date']) , 'death'] = 0
obs_df.loc[pd.isnull(obs_df['death_date'])==False , 'death'] = 1

In [0]:
deathdrop=['death_date','death_datetime','death_type_concept_id','cause_source_concept_id']
obs_df=obs_df.drop(columns=deathdrop)

In [0]:
obs_df.head(5)

,person_id,visit_occurrence_id,year_of_birth,ethnicity_concept_id,month_of_birth,day_of_birth,race_concept_id,gender_concept_id,birth_datetime,provider_id,visit_start_datetime,visit_concept_id,visit_type_concept_id,value_as_concept_id,observation_source_concept_id,value_as_string,observation_concept_id,observation_id,observation_datetime,death,obs_od
0,5,4.21363e+06,1935,38003564,9,1,8557,8507,1935-09-01T00:00:00.000Z,68338.0,2009-01-09T00:00:00.000Z,9202,4.48185e+07,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,41,1.65693e+06,1941,38003564,2,1,8527,8532,1941-02-01T00:00:00.000Z,179231.0,2008-10-09T00:00:00.000Z,0,4.48185e+07,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,62,3.96316e+06,1943,38003564,9,1,8516,8507,1943-09-01T00:00:00.000Z,189926.0,2008-06-30T00:00:00.000Z,0,4.48185e+07,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,103,58348,1935,38003564,6,1,8657,8507,1935-06-01T00:00:00.000Z,42069.0,2009-07-25T00:00:00.000Z,0,4.48185e+07,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,125,1.88664e+06,1926,38003564,5,1,8657,8507,1926-05-01T00:00:00.000Z,54829.0,2009-01-13T00:00:00.000Z,0,4.48185e+07,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [0]:
len(obs_df)

764639

In [0]:
## change the types of columns
categorical= ['person_id','visit_occurrence_id','ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','value_as_string','observation_concept_id','observation_id'] ## 9 objects
for col in categorical:
    obs_df[col] = obs_df[col].astype('object')
obs_df['death'] = obs_df['death'].astype(int)

In [0]:
obs_df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

person_id                  97916
visit_occurrence_id        94834
ethnicity_concept_id           2
race_concept_id                6
gender_concept_id              2
birth_datetime               900
visit_start_datetime         878
visit_concept_id               3
visit_type_concept_id          1
value_as_string              129
observation_concept_id       837
observation_id            521834
observation_datetime        2739
dtype: int64

In [0]:
obs1 = obs_df

In [0]:
bos_type = obs1.groupby('observation_concept_id')['observation_concept_id'].agg({'count': len}).sort_values('count',ascending = False).head(20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


In [0]:
bos_type

<bound method DataFrame.count of                          count
observation_concept_id        
4015724.0               235588
440927.0                154704
4214956.0               152443
0.0                     144233
2614666.0                71361
4046550.0                42806
4058431.0                36918
2614669.0                33230
440922.0                 29891
4167217.0                27722
439405.0                 27253
4186037.0                25598
2614668.0                25489
4059467.0                23433
2720870.0                18349
40479553.0               17393
4090651.0                17204
4059356.0                15228
2614670.0                14099
4060089.0                10793>

In [0]:
type(obs_df['observation_concept_id'][1])

numpy.float64

In [0]:
names = [4015724.0, 440927.0, 4214956.0, 0.0, 2614666.0, 4046550.0, 4058431.0, 2614669.0, 440922.0, 4167217.0, 439405.0, 4186037.0, 2614668.0, 4059467.0, 2720870.0, 4090651.0, 40479553.0, 4059356.0, 2614670.0, 4060089.0]

In [0]:
obs_df = pd.DataFrame(obs_df)

In [0]:
type(obs_df['observation_concept_id'][:1:])

pandas.core.series.Series

In [0]:
obs_type = names
new_type = []
exist = []
for i in range(len(obs_df)):
   a = obs_df['observation_concept_id'][i]
   if a in obs_type:
     new_type.append(a)
     exist.append(1)
   else:
     new_type.append(None)
     exist.append(0)


In [0]:
obs_df['obs_type'] = new_type
obs_df['exist'] = exist

In [0]:
obs_df['obs_od']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
2495560   NaN
2495561   NaN
2495562   NaN
2495563   NaN
2495564   NaN
Name: obs_od, Length: 2495565, dtype: float64

In [0]:
table1 = obs_df[['person_id', 'obs_type','visit_start_datetime', 'exist']]
table1.head(20)

,person_id,obs_type,visit_start_datetime,exist
0,5,NaN,2009-01-09T00:00:00.000Z,0
1,41,NaN,2008-10-09T00:00:00.000Z,0
2,62,NaN,2008-06-30T00:00:00.000Z,0
3,103,NaN,2009-07-25T00:00:00.000Z,0
4,125,NaN,2009-01-13T00:00:00.000Z,0
5,190,NaN,2009-04-20T00:00:00.000Z,0
6,216,NaN,2008-08-09T00:00:00.000Z,0
7,221,NaN,2008-01-23T00:00:00.000Z,0
8,221,NaN,2009-08-19T00:00:00.000Z,0
9,225,NaN,2009-10-29T00:00:00.000Z,0


In [0]:
table2 = table1[table1['visit_start_datetime'] == table1.groupby('person_id')['visit_start_datetime'].transform('max')][['person_id', 'obs_type', 'exist']]
##get the latest visit as records

In [0]:
table3 = pd.unique(table2[['person_id', 'obs_type', 'exist']])

In [0]:
table2.head(20)

,person_id,obs_type,exist
4,125,NaN,0
5,190,NaN,0
9,225,NaN,0
19,512,NaN,0
21,573,NaN,0
23,618,NaN,0
47,971,NaN,0
58,1159,NaN,0
77,1242,NaN,0
93,1627,NaN,0


In [0]:
exist = []
table1.loc[pd.isnull(table1['obs_od']) , 'exist'] = 0
table1.loc[pd.isnull(table1['obs_od'])==False , 'exist'] = 1

In [0]:
table3 = table2.dropna()

In [0]:
table3[['person_id', 'obs_type']] = table2.groupby(['person_id', 'obs_type']).transform('max').head(20)

,exist
4,NaN
5,NaN
9,NaN
19,NaN
21,NaN
23,NaN
47,NaN
58,NaN
77,NaN
93,NaN


In [0]:
table3.pivot(values='exist', index='person_id', columns='obs_type')

In [0]:
import datetime
obs_df['visit_start_datetime'] = obs_df['visit_start_datetime'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.000Z"))
obs_df['birth_datetime'] = obs_df['birth_datetime'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.000Z"))

In [0]:
drug_df['age'] = drug_df['visit_start_datetime']-drug_df['birth_datetime']
drug_df['age'] = drug_df['age'].dt.days.astype('int16')

## exploration

In [0]:
percent_missing = obs_df.isnull().sum() * 100 / len(obs_df)
missing_value_df = pd.DataFrame({'column_name': obs_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
person_id,person_id,0.000000
visit_occurrence_id,visit_occurrence_id,0.000080
year_of_birth,year_of_birth,0.000000
ethnicity_concept_id,ethnicity_concept_id,0.000000
month_of_birth,month_of_birth,0.000000
day_of_birth,day_of_birth,0.000000
race_concept_id,race_concept_id,14.403231
gender_concept_id,gender_concept_id,0.000000
birth_datetime,birth_datetime,0.000000
provider_id,provider_id,0.307425


In [0]:
obs_df = obs_df.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [0]:
obs_df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

birth_datetime               900
visit_start_datetime         878
observation_date             896
value_as_string              129
observation_source_value    1382
observation_datetime        2739
dtype: int64

### feature engineering

In [0]:
exp = obs_df[['quantity','days_supply','age']]
exp.head(5)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
### value data scaling
scaler = StandardScaler()
scaler.fit(drug_df[['value_as_number','days_supply','age']])
drug_df[['quantity','days_supply','age']] = scaler.transform(drug_df[['quantity','days_supply','age']])

In [0]:
def encode_onehot(df, cols):
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient ='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df[cols] = vec_data
    return df


In [0]:
## encoding
pd.get_dummies(drug_df, columns=categorical).head(5)

In [0]:
drug_df[categorical].head(5)

In [0]:
a = [0.013689, 0.009606]
a = np.asarray(a)
t_mean = (0.013689 + 0.009606)/2
std = np.sqrt(a.var(ddof = 1))
t_stat = (a[0] - a[1])/std
t_stat 

### about measurement info

In [0]:
## for measurement value in each unit
g = sns.FacetGrid(measure_df, col='unit_concept_id')
g.map(sns.boxplot, 'death','value_as_number')

In [0]:
drug_df.loc[drug_df['value_as_number']> 100000 , 'value_as_number'] =None ##delete the extreme values

In [0]:
sns.boxplot(x = 'death', y = 'value_as_number', data = measure_df)

In [0]:
type(measure_df.measurement_datetime)

In [0]:
measure_df['measure_date']= measure_df.measurement_datetime.dt.date

In [0]:
measure_df.to_csv("measure2.csv")